In [1]:
from src.data_utils.dataset import build_datset_train_val
from src.Transformers.transformers_prototype import build_transformer_model
from src.callbacks import get_predefine_callbacks
import optuna
import tensorflow as tf

2023-08-18 14:54:16.086102: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-18 14:54:16.722793: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-18 14:54:17.452406: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-18 14:54:17.472041: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [2]:
train_dataset, val_dataset = build_datset_train_val(split=0.8, batch_size=256)

train split: 28160 | val split: 6656


In [3]:
MAX_LENGHT_SOURCE = 120
FEATURES_SIZE = 84
ATTENTION_HEADS = 8
DIM_EMBEDDINGS = 256

In [4]:
tf.zeros(shape=(120))

<tf.Tensor: shape=(120,), dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.], dtype=float32)>

In [5]:
source = tf.random.uniform(shape=(1, MAX_LENGHT_SOURCE-1, FEATURES_SIZE))

last_row_zero = tf.zeros(shape=(1, 1, 84))

padded_input = tf.concat([source, last_row_zero], axis=1)

In [6]:
padded_input.shape

TensorShape([1, 120, 84])

In [7]:
mask = tf.math.logical_not(tf.math.equal(padded_input, 0))

In [8]:
tf.math.equal(tf.reduce_sum(padded_input, axis=2), 0)

<tf.Tensor: shape=(1, 120), dtype=bool, numpy=
array([[False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False,  True]])

In [9]:
from src.custom.layers import LandmarkEmbedding
from src.Transformers.Encoder import TransformerEncoder
from src.Transformers.Decoder import TransformerDecoder
from src.Transformers.PositionalEncoding import BasicPositionalEmbeddings

MAX_LENGHT_SOURCE = 120
FEATURES_SIZE = 84
ATTENTION_HEADS = 8
DIM_EMBEDDINGS = 256

source = tf.random.uniform(shape=(1, MAX_LENGHT_SOURCE, FEATURES_SIZE))
source_emb = LandmarkEmbedding(embedings_dim=DIM_EMBEDDINGS, max_seq_length=MAX_LENGHT_SOURCE)(source)

print(source_emb.shape)

# mask = tf.math.equal(tf.reduce_sum(padded_input, axis=2), 0)

# ENCODER START
# Auto encoder
mha = tf.keras.layers.MultiHeadAttention(
                                    num_heads=1,
                                    key_dim=64,
                                    # value_dim=64,
                                    # attention_axes=(2,1)#(1,2),
                                    # output_shape=(MAX_LENGHT_SOURCE, FEATURES_SIZE)
)

# encoder = TransformerEncoder(num_heads=ATTENTION_HEADS, dim_emb=FEATURES_SIZE, dim_dense=FEATURES_SIZE)
# (query=inputs, key=inputs, value=inputs, attention_mask=mask)
encoded_source_sequence = mha(source_emb, source_emb)
# encoded_source_sequence = mha(query=source_emb, key=source_emb, value=source_emb, attention_mask=mask)
# # ENCODER END

print(encoded_source_sequence.shape)

2023-08-18 14:54:20.419909: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8700
2023-08-18 14:54:20.833346: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


(1, 120, 256)
(1, 120, 256)


In [10]:
TARGET_MAX_LENGHT=32
VOCAB_SIZE=65
# ENCODER START
# Auto encoder
source = tf.random.uniform(shape=(1, 120, 84))
source_emb = LandmarkEmbedding(embedings_dim=512, max_seq_length=120)(source)

encoder = TransformerEncoder(num_heads=8, dim_emb=512, dim_dense=64)
encoded_source_sequence = encoder(source_emb)    
# ENCODER END

In [11]:
# DECODER START
target = tf.keras.layers.Input(shape=(32,), dtype=tf.int32, name="target")

target_emb = BasicPositionalEmbeddings(max_tokens=65, max_seq_length=65+1, dim_emb=512)(target)
decoder = TransformerDecoder(num_heads=8, dim_emb=512, dim_dense=64)


decoder.get_casual_attention_mask(target_emb)
decoded_target_sequence = decoder(encoded_source_sequence, target_emb)
# decoded_target_sequence = tf.keras.layers.Dropout(learning_rate=trial.suggest_float("drop_out", 0.0, 0.5, log=True))(decoded_target_sequence)
# DECODER END

In [12]:
def objective(trial):
    tf.keras.backend.clear_session()
    
    model = build_transformer_model(trial=trial)
    model.fit(train_dataset, validation_data=val_dataset, epochs=20, callbacks=get_predefine_callbacks(model_name="v2"))

    val_loss = model.evaluate(val_dataset)[0]
    return  val_loss

In [13]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50, gc_after_trial=True, show_progress_bar=True)

[I 2023-08-18 14:54:21,544] A new study created in memory with name: no-name-f462db80-3405-486a-8f14-5778824b3fe7


  0%|          | 0/50 [00:00<?, ?it/s]

/home/matias/pythonenv/fingerspelling/lib/python3.10/site-packages/optuna/distributions.py:700: UserWarning: The distribution is specified by [1, 8] and step=2, but the range is not divisible by `step`. It will be replaced by [1, 7].
  warnings.warn(


Epoch 1/20


2023-08-18 14:54:25.057063: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55850c935720 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-18 14:54:25.057095: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 SUPER, Compute Capability 7.5
2023-08-18 14:54:25.250668: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-08-18 14:54:25.435852: W tensorflow/compiler/tf2xla/kernels/random_ops.cc:57] Warning: Using tf.random.uniform with XLA compilation will ignore seeds; consider using tf.random.stateless_uniform instead if reproducible behavior is desired. model/dropout/dropout/random_uniform/RandomUniform
2023-08-18 14:54:25.445784: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:38] Ignoring Assert operator sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWi

    164/Unknown - 70s 334ms/step - loss: 2.5683 - accuracy: 0.4196

In [ ]:
tf.keras.backend.clear_session()

trials = study.best_trials

for index, trial in enumerate(trials):
    print(f"Best model: {index+1}")
    model_name = "v2"

    model = build_gru_model(trial=trial)
    model.fit(train_dataset, validation_data=val_dataset, epochs=5000, callbacks=get_predefine_callbacks(model_name=model_name))
    print(model.summary())

    print('validation loss: {}'.format(trial.value))
    print("Best hyperparameters: {}".format(trial.params))

    model.load_weights(f"../best_model/prototype/{model_name}")

    print(f"Score in Validation: {model.evaluate(val_dataset)}")